In [ ]:
import csv
import cv2
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import SimpleITK as sitk
from scipy.ndimage import zoom
import scipy.ndimage as ndi
import imageio
import glob
%matplotlib inline
import math
import skimage
from skimage.measure import label,regionprops, perimeter
from skimage import measure, feature
import scipy.misc
from matplotlib import patches
#os.getcwd()
from collections import defaultdict
from sklearn.model_selection import train_test_split
from math import log
from skimage.filters import median
import matplotlib

### Basic functions for reading raw files

In [ ]:
def readCsv(csvfname):
    # read csv to list of lists
    with open(csvfname, 'r') as csvf:
        reader = csv.reader(csvf)
        csvlines = list(reader)
    return csvlines

In [ ]:
def readMhd(filename):
    # read mhd/raw image
    itkimage = sitk.ReadImage(filename)
    scan = sitk.GetArrayFromImage(itkimage) #3D image
    spacing = itkimage.GetSpacing() #voxelsize
    origin = itkimage.GetOrigin() #world coordinates of origin
    transfmat = itkimage.GetDirection() #3D rotation matrix
    return scan, spacing, origin, transfmat

In [ ]:
def getImgWorldTransfMats(spacing,transfmat):
    # calc image to world to image transformation matrixes
    transfmat = np.array([transfmat[0:3],transfmat[3:6],transfmat[6:9]])
    for d in range(3):
        transfmat[0:3,d] = transfmat[0:3,d]*spacing[d]
    transfmat_toworld = transfmat #image to world coordinates conversion matrix
    transfmat_toimg = np.linalg.inv(transfmat) #world to image coordinates conversion matrix
    return transfmat_toimg,transfmat_toworld

In [ ]:
def convertToImgCoord(xyz,origin,transfmat_toimg):
    # convert world to image coordinates
    xyz = xyz - origin
    xyz = np.round(np.matmul(transfmat_toimg,xyz))    
    return xyz

### Confirmed nodules by at least 2 radiologists

In [ ]:
confirmed = pd.read_csv('confirmed.csv', sep="\t", index_col='LNDbID')
print(confirmed.shape)
confirmed.head()

In [ ]:
confirmed.index

### Pre-processing pipeline 1

In [ ]:
MIN_BOUND = -1000.0 #HU values
MAX_BOUND = 400.0 #HU value
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1
    image[image<0] = 0
    image = (image*255).astype('uint8')
    return image

In [ ]:
PIXEL_MEAN = 0.25

def zero_center(image):
    image = image - PIXEL_MEAN
    image = (image).astype('uint8')
    return image

### Pre-processing pipeline 2

In [ ]:
def normalize_log(image):
    MIN_BOUND = -250
    MAX_BOUND = 1500
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1.
    image[image<0] = 0.
    image = (image*255).astype('uint8')
    return image

In [ ]:
def log_transform(image):
    c = 255 / log(1 + 255) 
    log_image = c * (np.log(image + 1))  
    log_image = np.array(log_image, dtype = np.uint8) 
    return log_image

### Pre-processing pipeline 3

In [ ]:
MIN_BOUND = -1000.0 #HU values
MAX_BOUND = 400.0 #HU value
    
def normalize(image):
    image = (image - MIN_BOUND) / (MAX_BOUND - MIN_BOUND)
    image[image>1] = 1
    image[image<0] = 0
    image = (image*255).astype('uint8')
    return image

In [ ]:
from skimage.filters import median

### Extracting 2D RGB images with nodule

#### Pipeline 1 extract images

In [ ]:
# Read nodules csv
csvlines = readCsv('Train\\trainset_csv\\trainNodules.csv')
header = csvlines[0]
nodules = csvlines[1:]

for n in nodules:
    lnd = int(n[header.index('LNDbID')])
    rad = int(n[header.index('RadID')])
    finding = int(n[header.index('FindingID')])
    
    if int(n[header.index('LNDbID')])==lnd and int(n[header.index('RadID')])==rad and int(n[header.index('FindingID')])==finding:
        ctr = np.array([float(n[header.index('x')]), float(n[header.index('y')]), float(n[header.index('z')])])
        class_label = np.array([int(n[header.index('Nodule')])])
        
        name = 'LNDb-{:04d}_finding{}_rad{}'.format(lnd,finding,rad)
    if name in confirmed.index:
    
        [scan,spacing,origin,transfmat] = readMhd('Train\\train\\LNDb-{:04}.mhd'.format(lnd))
        transfmat_toimg,transfmat_toworld = getImgWorldTransfMats(spacing,transfmat)
        ctr = convertToImgCoord(ctr,origin,transfmat_toimg)

        # 2D slice with nodule center
        scan_slice = scan[int(ctr[2])]
        scan_norm = normalize(scan_slice)
        scan_norm_zero = zero_center(scan_norm)

        # Previous slice
        scan_slice_previous = scan[int(ctr[2])-1]
        scan_norm_pre = normalize(scan_slice_previous)
        scan_norm_zero_pre = zero_center(scan_norm_pre)
        
        # Next slice
        scan_slice_next = scan[int(ctr[2])+1]
        scan_norm_next = normalize(scan_slice_next)
        scan_norm_zero_next = zero_center(scan_norm_next)
        
        # Resize the image
        width = 512
        height = 512
        dim = (height, width)
        resized = cv2.resize(scan_norm_zero, dim, interpolation = cv2.INTER_AREA)
        resized_previous = cv2.resize(scan_norm_zero_pre, dim, interpolation = cv2.INTER_AREA)
        resized_next = cv2.resize(scan_norm_zero_next, dim, interpolation = cv2.INTER_AREA)

        # Create a 3 channel RBG image
        R = np.stack((resized_previous, resized, resized_next), axis=2)
        
        # Save the 3 channel images as jpg                     
        #cv2.imwrite('yolov4/median/LNDb-{:04d}_finding{}_rad{}.jpg'.format(lnd,finding,rad), R)

#### Pipeline 2 extract images

In [ ]:

#Extract, pre-process and save 2D slice with nodule

# Read nodules csv
csvlines = readCsv('Train\\trainset_csv\\trainNodules.csv')
header = csvlines[0]
nodules = csvlines[1:]

for n in nodules:
    lnd = int(n[header.index('LNDbID')])
    rad = int(n[header.index('RadID')])
    finding = int(n[header.index('FindingID')])
    
    if int(n[header.index('LNDbID')])==lnd and int(n[header.index('RadID')])==rad and int(n[header.index('FindingID')])==finding:
        ctr = np.array([float(n[header.index('x')]), float(n[header.index('y')]), float(n[header.index('z')])])
        class_label = np.array([int(n[header.index('Nodule')])])
        
        name = 'LNDb-{:04d}_finding{}_rad{}'.format(lnd,finding,rad)
    if name in confirmed.index:

        [scan,spacing,origin,transfmat] = readMhd('Train\\train\\LNDb-{:04}.mhd'.format(lnd))
        transfmat_toimg,transfmat_toworld = getImgWorldTransfMats(spacing,transfmat)
        ctr = convertToImgCoord(ctr,origin,transfmat_toimg)

        # 2D slice with nodule center
        scan_slice = scan[int(ctr[2])]
        scan_norm = normalize_log(scan_slice)
        scan_norm_log = log_transform(scan_norm)

        # Previous slice
        scan_slice_previous = scan[int(ctr[2])-1]
        scan_norm_pre = normalize_log(scan_slice_previous)
        scan_norm_log_pre = log_transform(scan_norm_pre)
 
        # Next slice
        scan_slice_next = scan[int(ctr[2])+1]
        scan_norm_next = normalize_log(scan_slice_next)
        scan_norm_log_next = log_transform(scan_norm_next)
           
        # Resize the image
        width = 512
        height = 512
        dim = (height, width)
        resized = cv2.resize(scan_norm_log, dim, interpolation = cv2.INTER_AREA)
        resized_previous = cv2.resize(scan_norm_log_pre, dim, interpolation = cv2.INTER_AREA)
        resized_next = cv2.resize(scan_norm_log_next, dim, interpolation = cv2.INTER_AREA)

        # Create a 3 channel RBG image
        R = np.stack((resized_previous, resized, resized_next), axis=2)
        
        # Save the 3 channel images as jpg                     
        #cv2.imwrite('yolov4/median/LNDb-{:04d}_finding{}_rad{}.jpg'.format(lnd,finding,rad), R)

#### Pipeline 3 extract images

In [ ]:
#Extract, pre-process and save 2D slice with nodule

# Read nodules csv
csvlines = readCsv('Train\\trainset_csv\\trainNodules.csv')
header = csvlines[0]
nodules = csvlines[1:]

for n in nodules:
    lnd = int(n[header.index('LNDbID')])
    rad = int(n[header.index('RadID')])
    finding = int(n[header.index('FindingID')])
    
    if int(n[header.index('LNDbID')])==lnd and int(n[header.index('RadID')])==rad and int(n[header.index('FindingID')])==finding:
        ctr = np.array([float(n[header.index('x')]), float(n[header.index('y')]), float(n[header.index('z')])])
        class_label = np.array([int(n[header.index('Nodule')])])
        
        name = 'LNDb-{:04d}_finding{}_rad{}'.format(lnd,finding,rad)
    if name in confirmed.index:

        [scan,spacing,origin,transfmat] = readMhd('Train\\train\\LNDb-{:04}.mhd'.format(lnd))
        transfmat_toimg,transfmat_toworld = getImgWorldTransfMats(spacing,transfmat)
        ctr = convertToImgCoord(ctr,origin,transfmat_toimg)

        # 2D slice with nodule center
        scan_slice = scan[int(ctr[2])]
        scan_norm = normalize(scan_slice)
        scan_norm_med = median(scan_norm)

        # Previous slice
        scan_slice_previous = scan[int(ctr[2])-1]
        scan_norm_pre = normalize(scan_slice_previous)
        scan_norm_med_pre = median(scan_norm_pre)

        # Next slice
        scan_slice_next = scan[int(ctr[2])+1]
        scan_norm_next = normalize(scan_slice_next)
        scan_norm_med_next = median(scan_norm_next)
        
        # Resize the image
        width = 512
        height = 512
        dim = (height, width)
        resized = cv2.resize(scan_norm_med, dim, interpolation = cv2.INTER_AREA)
        resized_previous = cv2.resize(scan_norm_med_pre, dim, interpolation = cv2.INTER_AREA)
        resized_next = cv2.resize(scan_norm_med_next, dim, interpolation = cv2.INTER_AREA)

        # Create a 3 channel RBG image
        R = np.stack((resized_previous, resized, resized_next), axis=2)
        
        # Save the 3 channel images as jpg                     
        #cv2.imwrite('yolov4/median/LNDb-{:04d}_finding{}_rad{}.jpg'.format(lnd,finding,rad), R)

In [ ]:
def gray_conv(s):
    I8 = (((s - s.min()) / (s.max() - s.min())) * 255.9).astype(np.uint8)
    img = Image.fromarray(I8)
    return img

#Extract, pre-process and save 2D slice with nodule

# Read nodules csv
csvlines = readCsv('Train\\trainset_csv\\trainNodules.csv')
header = csvlines[0]
nodules = csvlines[1:]

for n in nodules:
    lnd = int(n[header.index('LNDbID')])
    rad = int(n[header.index('RadID')])
    finding = int(n[header.index('FindingID')])
    
    if int(n[header.index('LNDbID')])==lnd and int(n[header.index('RadID')])==rad and int(n[header.index('FindingID')])==finding:
        ctr = np.array([float(n[header.index('x')]), float(n[header.index('y')]), float(n[header.index('z')])])
        class_label = np.array([int(n[header.index('Nodule')])])
        
        name = 'LNDb-{:04d}_finding{}_rad{}.jpg'.format(lnd,finding,rad)
    if name in confirmed.index:

        [scan,spacing,origin,transfmat] = readMhd('Train\\train\\LNDb-{:04}.mhd'.format(lnd))
        transfmat_toimg,transfmat_toworld = getImgWorldTransfMats(spacing,transfmat)
        ctr = convertToImgCoord(ctr,origin,transfmat_toimg)

        # 2D slice with nodule center
        scan_slice = scan[int(ctr[2])]
        scan_slice_conv = gray_conv(scan_slice)
        
        # Previous slice
        scan_slice_previous = scan[int(ctr[2])-1]
        scan_slice_pre_conv = gray_conv(scan_slice_previous)
        
        # Next slice
        scan_slice_next = scan[int(ctr[2])+1]
        scan_slice_next_conv = gray_conv(scan_slice_next)
                
        R = np.stack((scan_slice_pre_conv, scan_slice_conv, scan_slice_next_conv), axis=2)
        img = Image.fromarray(R)
        img.save("yolov4/original/gray/LNDb-{:04d}_finding{}_rad{}.jpg".format(lnd,finding,rad))

### Coordinates annotations

In [ ]:
# This functions are used to find the new coordinates of the nodules after the (512,512) resize of the images
def toPercentage(img_orig, x1, y1, x2, y2, xc, yc):
    c, h, w = img_orig.shape
    x1p = x1 / w
    x2p = x2 / w
    y1p = y1 / h
    y2p = y2 / h
    xcp = xc / w
    ycp = yc / h
    return x1p, y1p, x2p, y2p, xcp, ycp

def toImCoord(x1p, y1p, x2p, y2p, xcp, ycp ):
    h,w = (512, 512)
    xmin = x1p * w
    xmax = x2p * w
    ymin = y1p * h
    ymax = y2p * h 
    xcenter = xcp * w
    ycenter = ycp * h
    return xmin, xmax, ymin, ymax, xcenter, ycenter

In [ ]:
# Absolte coordinates of all nodules regardless their size
def slice_info_wo(mask, slice_props, k):
    y1, x1, y2, x2 = slice_props[k].bbox #x1 is xmin; x2 is xmax
    yc_k, xc_k = slice_props[k].centroid

    x1p, y1p, x2p, y2p, xcp, ycp = toPercentage(mask, x1, y1, x2, y2, xc_k, yc_k)
    xmin, xmax, ymin, ymax, xcenter, ycenter = toImCoord(x1p, y1p, x2p, y2p, xcp, ycp)
                
    bh_k = ymax - ymin
    bw_k = xmax - xmin

    return xcenter, ycenter, bw_k, bh_k

In [ ]:
# Relative to the image size coordinates - yolo compatible
def slice_info_rel_wo(mask, slice_props, k):
    y1, x1, y2, x2 = slice_props[k].bbox #x1 is xmin; x2 is xmax
    yc_k, xc_k = slice_props[k].centroid

    x1p, y1p, x2p, y2p, xcp, ycp = toPercentage(mask, x1, y1, x2, y2, xc_k, yc_k)
    xmin, xmax, ymin, ymax, xcenter, ycenter = toImCoord(x1p, y1p, x2p, y2p, xcp, ycp)
                
    bh_k = ymax - ymin
    bw_k = xmax - xmin
        
    return xcp, ycp, bw_k/512, bh_k/512

In [ ]:
# Dictionary with all nodules' coordinates as values and the file name as a key
dic_wo = defaultdict(dict)
dic_rel_wo = defaultdict(dict)
for mask_file in glob.glob('Train\\masks\\*.mhd'):
    mask, spacing, origin, transfmat = readMhd(mask_file)
    props = regionprops(mask, intensity_image=None, cache=False, coordinates=None)

    for i in range(len(props)):
        labels = props[i].label
        zc, yc, xc = props[i].centroid

        slice_props = regionprops(mask[int(zc)][:][:]) # check if there is another nodule on the first nodule's center slice
                
        file = mask_file.replace('mhd','jpg').split('_')
                
        dic_wo[str('{}_finding{}_{}'.format(file[0],labels,file[1]))]={k+1: slice_info_wo(mask, slice_props, k) for k in range (len(slice_props))}
        dic_rel_wo[str('{}_finding{}_{}'.format(file[0],labels,file[1]))]={k+1: slice_info_rel_wo(mask, slice_props, k) for k in range (len(slice_props))}

#### Confirmed cases

In [ ]:
confirmed_positive = confirmed[confirmed.Nodule==1]
confirmed_negative = confirmed[confirmed.Nodule==0]
print(confirmed_positive.shape)
print(confirmed_negative.shape)

In [ ]:
confirm_pos = confirmed_positive["LNDbID"].values
confirm_neg = confirmed_negative["LNDbID"].values

#### YOLO compatible .txt files with relative nodule coordinates

In [ ]:
for k2, v2 in dic_rel_wo.items():
    k2split = k2.rsplit(".")[0].rsplit('\\')[2]
    if k2split in confirm_pos and v2[1] != None:
        f= open(k2split  + ".txt", "w")
        for k, v in v2.items():
            if k == 1 and v != None:
                f.write(f"{0} {v[0]} {v[1]} {v[2]} {v[3]}")
            if k > 1 and v != None:
                f.write(f"\n{0} {v[0]} {v[1]} {v[2]} {v[3]}")
        f.close()

In [ ]:
for k2, v2 in dic_rel_wo.items():
    k2split = k2.rsplit(".")[0].rsplit('\\')[2]
    if k2split in confirm_neg:
        f= open(k2split  + ".txt", "w")
        f.write("")
        f.close()

In [ ]:
print(np.mean(confirmed_positive.diameter.values))
print(np.min(confirmed_positive.diameter.values))
print(np.max(confirmed_positive.diameter.values))

In [ ]:
plt.hist(confirmed_positive.diameter.values, color='b', alpha=0.4)
plt.title("Distribution of nodules' size")
plt.xlabel("Size in milimeters")
plt.ylabel("Frequency")
plt.show()

#### Randomize patient wise

In [ ]:
confirmed_positive = confirmed_positive.reset_index()
confirmed_negative = confirmed_negative.reset_index()

In [ ]:
confirmed_positive["path"] = 'data/obj/' + confirmed_positive['LNDbID'] + ".jpg"
confirmed_negative["path"] = 'data/obj/' + confirmed_negative['LNDbID'] + ".jpg"

#### Group shuffle split

In [ ]:
from sklearn.model_selection import GroupShuffleSplit

X = np.array(confirmed_positive["path"])
y = np.array(confirmed_positive['Nodule'])
groups = np.array(confirmed_positive["id"])

gss = GroupShuffleSplit(n_splits=1, train_size=0.8, random_state=42)
gss.get_n_splits()

for train_idx, test_idx in gss.split(X, y, groups):
       print("TRAIN:", train_idx, "TEST:", test_idx)

In [ ]:
train = confirmed_positive.iloc[train_idx]
test = confirmed_positive.iloc[test_idx]
print(train.shape)
print(test.shape)

#### Save the train/test filenames in yolo compatible format

In [ ]:
train_txt = train['path'].values
np.savetxt('train.txt', train_txt, fmt='%s', delimiter=',')
test_txt = test['path'].values
np.savetxt('test.txt', test_txt, fmt='%s', delimiter=',')

In [ ]:
train_txt[:10]

### Visualizations

In [ ]:
scan = readMhd('Train\\train\\LNDb-0024.mhd')[0]
mask = readMhd('Train\\masks\\LNDb-0024_rad3.mhd')[0]
props = regionprops(mask, intensity_image=None, cache=False, coordinates=None)

for i in range(len(props)):
    if i == 0:    
        labels = props[i].label
        print("Nodule: ", labels)
        zc, yc, xc = props[i].centroid
        zmin, y1, x1, zmax, y2, x2 = props[i].bbox #x1 is xmin; x2 is xmax
        bw = x2 - x1
        bh = y2 - y1
        max_diff = max(zmax-zmin, y2-y1, x2-x1)
        xy =(x1,y1)
        print("Nodule coord: ", int(xc), int(yc), int(zc))
        d = props[i].equivalent_diameter
        print("Diameter: ", round(d))

        fig, ax = plt.subplots(1,2, figsize=(10,5))
        ax[0].imshow(mask[int(zc)])
        ax[0].add_patch(matplotlib.patches.Rectangle(xy, bw, bh, fill=False, ec = 'red'))
        ax[1].imshow(scan[int(zc)]) #scan[int(zc)]
        ax[1].add_patch(matplotlib.patches.Rectangle(xy, bw, bh, fill=False, ec = 'red'))

        slice_props = regionprops(mask[int(zc)][:][:]) # check if there is another nodule on the first nodule's center slice
        print("Number of subnodules on the same slice: ", len(slice_props) - 1)
        print()         
    plt.show()

In [ ]:
scan = readMhd('Train\\train\\LNDb-0011.mhd')[0]
mask = readMhd('Train\\masks\\LNDb-0011_rad3.mhd')[0]
props = regionprops(mask, intensity_image=None, cache=False, coordinates=None)
print("Number of nodules: ", len(props))
print()
for i in range(len(props)):
    if i == 0:    
        labels = props[i].label
        print("Nodule: ", labels)
        zc, yc, xc = props[i].centroid
        zmin, y1, x1, zmax, y2, x2 = props[i].bbox #x1 is xmin; x2 is xmax
        bw = x2 - x1
        bh = y2 - y1
        max_diff = max(zmax-zmin, y2-y1, x2-x1)
        xy =(x1,y1)
        print("Nodule coord: ", int(xc), int(yc), int(zc))
        d = props[i].equivalent_diameter
        print("Diameter: ", round(d))

        fig, ax = plt.subplots(1,2, figsize=(10,5))
        fig.tight_layout()
        ax[0].imshow(mask[int(zc)])
        ax[0].add_patch(matplotlib.patches.Rectangle(xy, bw, bh, fill=False, ec = 'red'))
        ax[0].axis('off')
        ax[0].set_title("Slice mask holding 1 nodule centroid & 2 more nodules")
        ax[1].imshow(scan[int(zc)]) #scan[int(zc)]
        ax[1].add_patch(matplotlib.patches.Rectangle(xy, bw, bh, fill=False, ec = 'red'))
        ax[1].axis('off')
        ax[1].set_title("Raw slice with corresponding mask nodules")

        slice_props = regionprops(mask[int(zc)][:][:]) # check if there is another nodule on the first nodule's center slice
        print("Number of subnodules on the same slice: ", len(slice_props) - 1)
        print()

        if len(slice_props) > 0:
            for k in range(1,len(slice_props)):
                d2 = slice_props[k].equivalent_diameter
                print("Sub nodule diameter: ", round(d2))
                if d2 > 0: #only greather than 3mm
                    ymin, xmin, ymax, xmax = slice_props[k].bbox #x1 is xmin; x2 is xmax
                    yc_k, xc_k = slice_props[k].centroid
                    xy_k = (xmin, ymin)
                    bh_k = ymax - ymin
                    bw_k = xmax - xmin
                    ax[0].add_patch(matplotlib.patches.Rectangle(xy_k, bw_k, bh_k, fill=False, ec = 'white'))
                    ax[1].add_patch(matplotlib.patches.Rectangle(xy_k, bw_k, bh_k, fill=False, ec = 'white'))
                    print("Properties x,y,w,h: ", int(xc_k), int(yc_k), int(bw_k), int(bh_k))
                    print()
        plt.show()

In [ ]:
def extractCube(scan,spacing,xyz,cube_size=80,cube_size_mm=51):
    # Extract cube of cube_size^3 voxels and world dimensions of cube_size_mm^3 mm from scan at image coordinates xyz
    xyz = np.array([xyz[i] for i in [2,1,0]],np.int)
    spacing = np.array([spacing[i] for i in [2,1,0]])
    scan_halfcube_size = np.array(cube_size_mm/spacing/2,np.int)
    if np.any(xyz<scan_halfcube_size) or np.any(xyz+scan_halfcube_size>scan.shape): # check if padding is necessary
        maxsize = max(scan_halfcube_size)
        scan = np.pad(scan,((maxsize,maxsize,)),'constant',constant_values=0)
        xyz = xyz+maxsize
    
    scancube = scan[xyz[0]-scan_halfcube_size[0]:xyz[0]+scan_halfcube_size[0], # extract cube from scan at xyz
                    xyz[1]-scan_halfcube_size[1]:xyz[1]+scan_halfcube_size[1],
                    xyz[2]-scan_halfcube_size[2]:xyz[2]+scan_halfcube_size[2]]

    sh = scancube.shape
    scancube = zoom(scancube,(cube_size/sh[0],cube_size/sh[1],cube_size/sh[2]),order=2) #resample for cube_size
    
    return scancube

In [ ]:
csvlines = readCsv('Train\\trainset_csv\\trainNodules.csv')
header = csvlines[0]
nodules = csvlines[1:2]

for n in nodules:
    lnd = int(n[header.index('LNDbID')])
    rad = int(n[header.index('RadID')])
    finding = int(n[header.index('FindingID')])

    [scan,spacing,origin,transfmat] =  readMhd('Train\\train\\LNDb-{:04}.mhd'.format(lnd))
    [mask,spacing,origin,transfmat] =  readMhd('Train\\masks\\LNDb-{:04}_rad{}.mhd'.format(lnd,rad))


    if int(n[header.index('LNDbID')])==lnd and int(n[header.index('RadID')])==rad and int(n[header.index('FindingID')])==finding:
        ctr = np.array([float(n[header.index('x')]), float(n[header.index('y')]), float(n[header.index('z')])])
        
    
    # Convert coordinates to image
        transfmat_toimg,transfmat_toworld = getImgWorldTransfMats(spacing,transfmat)
        ctr = convertToImgCoord(ctr,origin,transfmat_toimg)
        scan_cube = extractCube(scan,spacing,ctr)
        scan_cube_central = normalize(scan_cube[int(scan_cube.shape[0]/2)])
        mask[mask!=finding] = 0
        mask[mask>0] = 1
        mask_cube = extractCube(mask,spacing,ctr)
        plt.figure(figsize = (8,3))
        plt.imshow(scan_cube_central, cmap='gray')
        plt.axis("off")
        plt.show()

In [ ]:
csvlines = readCsv("yolov4\\data\\negative\\comp4_det_test_nodule_median.txt")
predic = csvlines[0:]

In [ ]:
img_to_plot = []
for line in predic:
    line = line[0].split(" ")
    name = line[0] + ".jpg"
    if name not in img_to_plot: 
        img_to_plot.append(name) 

In [ ]:
for k,v in dic_wo.items():    
    k = k.split("\\")[2]
    if k in img_to_plot:
        print(k)
        fig, ax = plt.subplots(1,1, figsize=(20,5))
        img = plt.imread("yolov4/median/negative/{}".format(k))
        ax.imshow(img)
        for k2, v2 in v.items():
            if v2 != None:
                #print(v2)
                xy = (v2[0]-(v2[2])/2, v2[1]-(v2[3]/2))
                #print(xy)
                bw = v2[2]
                bh = v2[3]
                ax.add_patch(matplotlib.patches.Rectangle(xy, bw, bh, fill=False, ec = 'yellow', lw = 1)) # ground truth
        for line in predic:
            line = line[0].split(" ")
            name = line[0] + ".jpg"
            if name == k and float(line[1]) >= 0.5:
                w = (float(line[4]) - float(line[2]))
                h = (float(line[5]) - float(line[3]))
                xyp = (float(line[2]), float(line[3]))
                ax.add_patch(matplotlib.patches.Rectangle(xyp, w, h, fill=False, ec = 'magenta', lw = 1))           
    plt.show()